# Introduction

The [TinyLlama](https://github.com/jzhang38/TinyLlama) project "aims to pretrain a 1.1B Llama model on 3 trillion tokens." 1.1B tokens represents a considerable step up from the small GPT model we [previously fine-tuned](../2_gpt2_single_gpu/2.%20GPT2%20on%20a%20single%20GPU.ipynb). That model had 124M parameters; TinyLlama, while still small by the standards of most widely-used LLMs, is almost ten times the size. We will need around 20GB VRAM at a bare minimum to fine-tune this model.

## Instruction Tuning
We are going to focus on instruction tuning in this example. Instruction Tuning is a supervised learning technique in which we train the model on instruction/output pairs with the goal of training the model to follow human instructions. Before instruction tuning, the base model is trained on next-token completion. We saw this in the GPT2 example: we provided the start of a story and the model completed it. An instruction-tuned model, on the other hand, is trained to answer a question or instruction.

[This repository](https://github.com/xiaoya-li/Instruction-Tuning-Survey) contains a wealth of information on the current state of the field of instruction tuning.

The [TinyLlama repository](https://github.com/jzhang38/TinyLlama/tree/main/sft) includes scripts for fine-tuning. While these will be useful references, we will try to proceed with an approach similar to that used in the gpt2 and t5-small notebooks--purely for the sake of making this notebook a reasonable learning step following those.

# The Data
We will use the [SlimOrca](https://huggingface.co/datasets/Open-Orca/SlimOrca) dataset. This is a curated subset of the much larger [OpenOrca](https://huggingface.co/datasets/Open-Orca/OpenOrca) dataset. Why this dataset? It's one of the most popular sources of instruction data on Hugging Face, and its size is more manageable than the full OpenOrca dataset. That's all!

# 1. Load the model and try some examples

We'll begin, as always, by loading the model and trying out some examples.

In [ ]:
%pip install --upgrade -r ./tinyllama_requirements.txt

In [1]:
# Some Environment Setup
OUTPUT_DIR = "../results/TinyLlama/" # the path to the output directory; where model checkpoints will be saved
LOG_DIR = "../logs/TinyLlama/" # the path to the log directory; where logs will be saved
CACHE_DIR = "../cache/TinyLlama/" # the path to the cache directory; where cache files will be saved

In [62]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

model_ckpt = "TinyLlama/TinyLlama-1.1B-intermediate-step-1431k-3T"

tokenizer = AutoTokenizer.from_pretrained(
    model_ckpt,
)

tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_ckpt,
    device_map="auto",
)

# Inference
def generate(prompt, max_new_tokens=100):
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(model.device)
    gen_tokens = model.generate(input_ids, max_new_tokens=max_new_tokens,
                                eos_token_id=tokenizer.eos_token_id,
                                repetition_penalty=1.1)
    return tokenizer.batch_decode(gen_tokens, skip_special_tokens=True)[0]

print(generate("Here are step-by-step instructions to make a great cup of coffee with a Chemex coffee maker:\n1."))

Here are step-by-step instructions to make a great cup of coffee with a Chemex coffee maker:
1. Prepare the grounds
2. Add water
3. Grind the beans
4. Pour the water into the pot
5. Let it brew
6. Enjoy your coffee!
What is a Chemex?
A Chemex is a type of coffee maker that uses a ceramic filter to extract the flavor from the coffee beans. The Chemex was invented by a man named John C. Hunt in 1908


In this example, we structured our prompt with completion in mind: we generated the first part of the full text and asked the model to complete it. What happens if, instead, we ask a question or give an instruction?

In [3]:
# Question
print(generate("How do I make coffee with a Chemex coffee maker?"))

How do I make coffee with a Chemex coffee maker?
How do you make coffee with a Chemex?
What is the best way to brew coffee with a Chemex?
How do you make coffee with a Chemex filter?
How do you make coffee with a Chemex filter and a Chemex?
How do you make coffee with a Chemex filter and a Chemex filter?
How do you make coffee with a Chemex filter and a Chemex filter?
How do you make coffee with a Chemex filter and


In [4]:
# Instruction
print(generate("Tell me how to make coffee with a Chemex coffee maker."))

Tell me how to make coffee with a Chemex coffee maker.
I'm not sure if you can get the Chemex in the UK, but I think it's worth a try.
The Chemex is a great coffee maker and I love mine!
I have one of these and I love it. It makes a great cup of coffee every time.
I have a Chemex and I love it. It makes a great cup of coffee every time.
I have a Chemex and I love it. It makes a great cup


These did not work because the model has not been instruction tuned. Our task is to change that!

# 2. Getting and Exploring the Data

In [5]:
from datasets import load_dataset
from pathlib import Path

slimorca = load_dataset('Open-Orca/SlimOrca',
                           cache_dir=str(Path(CACHE_DIR) / "data"))


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [6]:
import json
print(json.dumps(slimorca["train"][0], indent=4))

{
    "conversations": [
        {
            "from": "system",
            "value": "You are an AI assistant. You will be given a task. You must generate a detailed and long answer.",
            "weight": null
        },
        {
            "from": "human",
            "value": "Write an article based on this \"A man has been charged with murder and attempted murder after a woman and the man she was on a date with were stabbed at a restaurant in Sydney, Australia.\"",
            "weight": 0.0
        },
        {
            "from": "gpt",
            "value": "Title: Tragedy Strikes in Sydney: Victims Stabbed during Dinner Date, Suspect Charged with Murder and Attempted Murder\n\nIntroduction\n\nA seemingly ordinary evening took a tragic turn in Sydney, Australia when a romantic dinner date at a local restaurant ended in bloodshed, as a man allegedly attacked a couple, stabbing both victims. Local authorities have charged the suspect with murder and attempted murder in connectio

You'll see that there are three components to the sample entry:
1. A *system message*: this should be familiar if you've used e.g. ChatGPT via the API. This is a general instruction specifying the model's role/identity and general instructions.
2. A *human message*: this is the specific instruction passed to the model by a human.
3. a *gpt*: this is the AI model's response.

So we want to use this dataset to fine-tune the model such that it will respond more like the *gpt* message when given the *system* and *human* messages.

# 3. Formatting the Data

First, we need to get these entries into a format we can actually use for fine-tuning. [Appendix A](#Appendix-A:-Looking-at-the-tinyllama-fine-tuning-code) digs into the fine tuning code from the TinyLlama repo to see how the authors handled data formatting. We're going to take a slightly different approach and use the [chat model templates](https://huggingface.co/docs/transformers/main/en/chat_templating#templates-for-chat-models) from the Transformers library. The Hugging Face docs recommend applying the chat templates as a preprocessing step. Let's take a look at how they work.

## Transformers Chat Templates
Chat Templates are attributes of tokenizers. If a chat template isn't set explicitly, the default template for that model class is used. Let's see if there is a chat template set here.


In [7]:
print(tokenizer.chat_template), print(tokenizer.default_chat_template)


No chat template is defined for this tokenizer - using the default template for the LlamaTokenizerFast class. If the default is not appropriate for your model, please set `tokenizer.chat_template` to an appropriate template. See https://huggingface.co/docs/transformers/main/chat_templating for more information.



None
{% if messages[0]['role'] == 'system' %}{% set loop_messages = messages[1:] %}{% set system_message = messages[0]['content'] %}{% elif false == true and not '<<SYS>>' in messages[0]['content'] %}{% set loop_messages = messages %}{% set system_message = 'You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.\n\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don\'t know the answer to a question, please don\'t share false information.' %}{% else %}{% set loop_messages = messages %}{% set system_message = false %}{% endif %}{% for message in loop_messages %}{% if (message['role'] == 'user') != (loop.index0 % 2 == 0) %}{{ raise_exception('Conversation roles must

(None, None)

There is no chat template defined for this tokenizer, so we'll use the default LlamaTokenizerFast class default template. To populate this template, we apply the template to this standard chat format.

In [8]:
chat = [
    {"role": "system", "content": "You are a helpful assistant and an expert at making coffee."},
    {"role": "user", "content": "How do I make coffee with a Chemex coffee maker?"},
    {"role": "assistant", "content": "To make coffee with a Chemex:\n1. Boil water to about 200°F (93°C).\n2. Place the Chemex filter in the top and rinse it with hot water to remove paper taste and warm the vessel. Discard the rinse water.\n3. Add coffee grounds to the filter. Use a medium-coarse grind, about 1 gram of coffee per 16 grams of water.\n4. Pour just enough hot water to saturate the grounds. Wait 30 seconds for the coffee to 'bloom'.\n5. Slowly pour the remaining water over the grounds in a circular motion. Aim for a total brew time of 3.5 to 4.5 minutes.\n6. Once brewing is complete, remove the filter and enjoy."}
]

In [9]:
print(tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=False))

<s>[INST] <<SYS>>
You are a helpful assistant and an expert at making coffee.
<</SYS>>

How do I make coffee with a Chemex coffee maker? [/INST] To make coffee with a Chemex:
1. Boil water to about 200°F (93°C).
2. Place the Chemex filter in the top and rinse it with hot water to remove paper taste and warm the vessel. Discard the rinse water.
3. Add coffee grounds to the filter. Use a medium-coarse grind, about 1 gram of coffee per 16 grams of water.
4. Pour just enough hot water to saturate the grounds. Wait 30 seconds for the coffee to 'bloom'.
5. Slowly pour the remaining water over the grounds in a circular motion. Aim for a total brew time of 3.5 to 4.5 minutes.
6. Once brewing is complete, remove the filter and enjoy. </s>


And because we're trying to train on input/output pairs (following the TinyLlama fine tuning code examples), we'll split this at the `[/INST]` and have the input as everything up to and including the `[/INST]` and the output as everything after (including the space!). Let's write a method to do that.

In [10]:
ex = slimorca["train"][0]
ex['conversations'][2]

{'from': 'gpt',
 'value': "Title: Tragedy Strikes in Sydney: Victims Stabbed during Dinner Date, Suspect Charged with Murder and Attempted Murder\n\nIntroduction\n\nA seemingly ordinary evening took a tragic turn in Sydney, Australia when a romantic dinner date at a local restaurant ended in bloodshed, as a man allegedly attacked a couple, stabbing both victims. Local authorities have charged the suspect with murder and attempted murder in connection with the horrific crime. In this article, we delve into the chilling incident, the suspect, and the impact it has had on the community.\n\nThe Incident\n\nThe alarming event took place at a popular restaurant in Sydney, Australia. A woman and her date were enjoying their meal – blissfully unaware of the tragic circumstances that would soon unfold. As they shared conversation and indulged in their dinner, a man approached the couple and, without warning, began to stab them with a sharp object.\n\nPatrons at the restaurant scrambled for safe

In [11]:
def format_slimorca(ex, tokenizer):
    role_mapping = {"gpt": "assistant", "system": "system", "human": "user"}
    chat = [
        {"role": role_mapping[message["from"]], "content": message["value"]}
        for message in ex["conversations"]
    ]
    fmt_chat = tokenizer.apply_chat_template(
        chat, tokenize=False, add_generation_prompt=False
    )
    split_chat = fmt_chat.split("[/INST]")
    return split_chat[0] + "[/INST]", split_chat[1]


# Map to the dataset
slimorca["train"] = slimorca["train"].map(
    lambda ex: {
        "input": format_slimorca(ex, tokenizer)[0],
        "output": format_slimorca(ex, tokenizer)[1],
    }
)

Map:   0%|          | 0/517982 [00:00<?, ? examples/s]

In [12]:
slimorca = slimorca.remove_columns(
            [col for col in slimorca.column_names['train'] if col not in ['input', 'output']]
        )

slimorca["train"][0]

{'input': '<s>[INST] <<SYS>>\nYou are an AI assistant. You will be given a task. You must generate a detailed and long answer.\n<</SYS>>\n\nWrite an article based on this "A man has been charged with murder and attempted murder after a woman and the man she was on a date with were stabbed at a restaurant in Sydney, Australia." [/INST]',
 'output': " Title: Tragedy Strikes in Sydney: Victims Stabbed during Dinner Date, Suspect Charged with Murder and Attempted Murder\n\nIntroduction\n\nA seemingly ordinary evening took a tragic turn in Sydney, Australia when a romantic dinner date at a local restaurant ended in bloodshed, as a man allegedly attacked a couple, stabbing both victims. Local authorities have charged the suspect with murder and attempted murder in connection with the horrific crime. In this article, we delve into the chilling incident, the suspect, and the impact it has had on the community.\n\nThe Incident\n\nThe alarming event took place at a popular restaurant in Sydney, 

And now we will tokenize the inputs and the outputs. We could combine this with the above step, but we'll leave it separate for transparency.

In [82]:
def tokenize_function(examples):
    return {
        "input_ids": tokenizer(examples["input"], padding=True, truncation=True, max_length=1024)["input_ids"],
        "labels": tokenizer(examples["output"], padding=True, truncation=True, max_length=256)["input_ids"]
    }

slimorca_tokenized = slimorca.map(tokenize_function, batched=True)

Map:   0%|          | 0/517982 [00:00<?, ? examples/s]

In [86]:
for i in range(3):
    print("Length of input_ids:", len(slimorca_tokenized["train"][i]['input_ids'])) # Adjust the dataset part ('train') as necessary
    print("Length of labels:", len(slimorca_tokenized["train"][i]['labels'])) # Adjust the dataset part ('train') as necessary


Length of input_ids: 1024
Length of labels: 256
Length of input_ids: 1024
Length of labels: 256
Length of input_ids: 1024
Length of labels: 256


Now, in the [gpt2 example](../2_gpt2_single_gpu/2.%20GPT2%20on%20a%20single%20GPU.ipynb), we will configure a *collator*. We can take some inspiration from the [collator defined in the TinyLlama fine-tuning script](https://github.com/jzhang38/TinyLlama/blob/11a02ce085c1670bd009e6d4385701ff06a7f6cf/sft/finetune.py#L252C19-L252C19).

In [87]:
from datasets import DatasetDict

# Split the tokenized dataset into training and validation sets
slimorca_tokenized_split = slimorca_tokenized['train'].train_test_split(test_size=0.1)

# Format the split datasets into a DatasetDict for compatibility with Hugging Face's Trainer
slimorca_tokenized_split = DatasetDict(
    {
        "train": slimorca_tokenized_split["train"],
        "valid": slimorca_tokenized_split["test"],
    }
)

slimorca_tokenized_split

DatasetDict({
    train: Dataset({
        features: ['input', 'output', 'input_ids', 'labels'],
        num_rows: 466183
    })
    valid: Dataset({
        features: ['input', 'output', 'input_ids', 'labels'],
        num_rows: 51799
    })
})

In [88]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False,
)

# 4. Fine-tune the model

In [ ]:
from transformers import TrainingArguments, Trainer
import mlflow

# Define the training arguments
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=1,
    per_device_train_batch_size=1, 
    per_device_eval_batch_size=1, 
    warmup_steps=1,
    weight_decay=0.01,
    logging_dir=LOG_DIR,
    logging_steps=10,  # Log every 10 steps
    evaluation_strategy="steps",  # Evaluate every 'eval_steps'
    eval_steps=10000,
    fp16=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=slimorca_tokenized_split["train"],
    eval_dataset=slimorca_tokenized_split["valid"],  # Use only the first 5k rows for eval data
    data_collator=data_collator,
)

# Start training and track with MLflow
with mlflow.start_run(log_system_metrics=True):
    trainer.train()
    mlflow.log_params(training_args.to_dict())

# Appendix A: Looking at the tinyllama fine-tuning code

You can fine the tinyllama fine-tuning code [here](https://github.com/jzhang38/TinyLlama/tree/main/sft). It's worth the time, at this phase of learning about fine-tuning, to read through it and learn about some of the approaches they use.

Let's first take a look at the [train()](https://github.com/jzhang38/TinyLlama/blob/11a02ce085c1670bd009e6d4385701ff06a7f6cf/sft/finetune.py#L492) method. It begins by using the [`HFArgumentParser`](https://huggingface.co/docs/transformers/v4.36.1/en/internal/trainer_utils#transformers.HfArgumentParser) to configure the training arguments. Earlier, the training code defined a number of `dataclass`es for e.g. training arguments, data arguments, etc. `HFArgumentParser` provides an approach for parsing command line arguments directly into instances of these dataclass types. So instead of simply defining arguments in notebook cells, as we've been doing, this approach provides a structured way to parse command line arguments. And, indeed, the repo provides a [shell script](https://github.com/jzhang38/TinyLlama/blob/main/sft/script.sh) for running the fine-tuning script with a defined set of arguments.

The [next major section](https://github.com/jzhang38/TinyLlama/blob/11a02ce085c1670bd009e6d4385701ff06a7f6cf/sft/finetune.py#L514C3-L514C3) of the training script is focused on preparing the data, using the [`make_data_module`](https://github.com/jzhang38/TinyLlama/blob/11a02ce085c1670bd009e6d4385701ff06a7f6cf/sft/finetune.py#L354) method defined earlier in the script. That method is set up to handle a few different potential fine-tuning data sources (slimorca is not included among them). It maps each of them to the expected format: an input string and an output string.

I found the handling of alpaca-formatted datasets instructive. The [alpaca](https://huggingface.co/datasets/tatsu-lab/alpaca?row=0) dataset includes instructions and optional inputs that follow a specified format. For examples with inputs, the format is:

```
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Input:
{input}

### Response:
```

The following code snippet in the TinyLlama repo handles this formatting (in the alpaca dataset, the inputs/outputs are not pre-formatted)

```python
ALPACA_PROMPT_DICT = {
    "prompt_input": (
        "Below is an instruction that describes a task, paired with an input that provides further context. "
        "Write a response that appropriately completes the request.\n\n"
        "### Instruction:\n{instruction}\n\n### Input:\n{input}\n\n### Response: "
    ),
    "prompt_no_input": (
        "Below is an instruction that describes a task. "
        "Write a response that appropriately completes the request.\n\n"
        "### Instruction:\n{instruction}\n\n### Response: "
    ),
}

def extract_alpaca_dataset(example):
    if example.get("input", "") != "":
        prompt_format = ALPACA_PROMPT_DICT["prompt_input"]
    else:
        prompt_format = ALPACA_PROMPT_DICT["prompt_no_input"]
    return {'input': prompt_format.format(**example)}
```

We're seeing some repeating patterns across training scripts (the examples so far in this repo and the TinyLlama code). Each fine-tuning run so far requires the following:
- process the data
- set up training arguments
- set up logging

An additional step, as we get to multi-gpu and multi-node setups, will be configuring devices and processses—-see the [script.sh](https://github.com/jzhang38/TinyLlama/blob/main/sft/script.sh) shell script from TinyLlama for an example, which uses [accelerate launch](https://huggingface.co/docs/accelerate/basic_tutorials/launch), a helper command that makes it easier to launch training scripts on different hardware.